# Computational Social Science Project #3
Group number: 1

Group members: Benjamin Fields, Ernesto Gutierrez, and Nehal Eldeeb

Semester: Fall 2021

## 1. Introduction

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelBinarizer

# Make sure to import other libraries that will be necessary for training models!

## 2. Data Pre-Processing & Cleaning

In [2]:
# Inspections Data 2011 - 2013
chicago_inspections_2011_to_2013 = pd.read_csv("data/Chicago Inspections 2011-2013.csv")

# Inspections Data 2014
chicago_inspections_2014 = pd.read_csv("data/Chicago Inspections 2014.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/Chicago Inspections 2011-2013.csv'

In [ ]:
# Look at the inspections data
chicago_inspections_2011_to_2013.head()

In [ ]:
# List column names
chicago_inspections_2011_to_2013.columns

In [ ]:
# Drop column names related to geography, identification, and pass/fail flags that perfectly predict the outcome
chicago_inspections_2011_to_2013.drop(columns = ['AKA_Name', 
                                                'License',
                                                'Address',
                                                'City',
                                                'State',
                                                'Zip',
                                                'Latitude',
                                                'Longitude',
                                                'Location',
                                                'ID',
                                                'LICENSE_ID',
                                                'LICENSE_TERM_START_DATE',
                                                'LICENSE_TERM_EXPIRATION_DATE',
                                                'LICENSE_STATUS',
                                                'ACCOUNT_NUMBER',
                                                'LEGAL_NAME',
                                                'DOING_BUSINESS_AS_NAME',
                                                'ADDRESS',
                                                'CITY',
                                                'STATE',
                                                'ZIP_CODE',
                                                'WARD',
                                                'PRECINCT',
                                                'LICENSE_CODE',
                                                'BUSINESS_ACTIVITY_ID',
                                                'BUSINESS_ACTIVITY',
                                                'LICENSE_NUMBER',
                                                'LATITUDE',
                                                'LONGITUDE',
                                                'pass_flag',
                                                'fail_flag'],
                                     inplace = True)

chicago_inspections_2011_to_2013.set_index(['Inspection_ID', 'DBA_Name'], inplace = True)

In [ ]:
# Convert the Inspection Date to a datetime format
chicago_inspections_2011_to_2013['Inspection_Date'] = pd.to_datetime(chicago_inspections_2011_to_2013['Inspection_Date'], infer_datetime_format=True)  

### Visualization

What do inspections look like over time?

In [ ]:
# Visualize Inspections Over Time
chicago_inspections_2011_to_2013['Inspection_MonthYear'] = chicago_inspections_2011_to_2013['Inspection_Date'].dt.to_period('M')
counts_by_day = chicago_inspections_2011_to_2013.groupby('Inspection_MonthYear').count().rename(columns = {'Facility_Type': 'Count'})['Count'].reset_index()
counts_by_day.set_index(["Inspection_MonthYear"], inplace = True)
counts_by_day.plot(title = "Inspections by Month and Year")

What do the results look like? 

In [ ]:
# Inspection Results
sns.catplot(data = chicago_inspections_2011_to_2013,
           x = "Results",
           kind = "count")

plt.title("Inspection Results")
plt.show()

What if we separate by facility type?

In [ ]:
# Inspection Results by Facility Type (Restaurant or Not)
sns.catplot(data = chicago_inspections_2011_to_2013,
           x = "Results",
           kind = "count",
           hue = 'Facility_Type_Clean')

plt.title("Inspection Results by Facility Type")
plt.show()

### Preprocess Data

In [ ]:
# Drop datetime info
chicago_inspections_2011_to_2013 = chicago_inspections_2011_to_2013.dropna().drop(['Inspection_Date',
                                      'minDate',
                                      'maxDate',
                                      'Inspection_MonthYear'],
                                      axis = 1)

In [ ]:
chicago_inspections_2011_to_2013

In [ ]:
# Set target variable. 
y = chicago_inspections_2011_to_2013['Results']
## Comment out the following code if you don't want to binarize the target variable
y = y.replace({'Pass w/ Conditions': 'Pass'})
lb_style = LabelBinarizer()
y = lb_style.fit_transform(y)
# Recode 0s and 1s so 1s are "Fail"
y = np.where(y == 1, 0 ,1)

# All other features in X
X = chicago_inspections_2011_to_2013.drop(columns = ['Results'])
X = pd.get_dummies(X)

In [ ]:
X.head()

## 3. Fit Models

In [ ]:
#This is importing the required packages and functions for running all three models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
%matplotlib inline
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [ ]:
#Set the seed
np.random.seed(12345)

### 3.1 Data Splitting

In [ ]:
#Create the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .80, test_size=0.20, stratify=y)
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, train_size = .75, test_size = .25, stratify = y_train)

### 3.2 Model 1 - Logistic Regression

Be sure to do the following:

1. Import the appropriate library from sklearn
2. Set up a hyperparameter grid (check out our previous labs to see how to do this)
3. Find the best hyperparameters, and then fit your model (using train/validation splits or cross-validation)

In [ ]:
#Logistic Regression Model
logit_reg = LogisticRegression()
logit_model = logit_reg.fit(X_train, y_train.ravel())
y_pred = logit_model.predict(X_validate)
#Handling the Data for the Logistic Regression Model
logit_data = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(logit_model.coef_))], axis = 1)
logit_data.columns = ['Feature', 'Coefficient']
logit_data['abs_coef'] = abs(logit_data['Coefficient'])
#Visualizing the Logistic Regression Model
sns.barplot(x="Coefficient", y="Feature", data=logit_data.nlargest(10, 'abs_coef')).set_title("Top Logit Coefficients")
plt.show()

In [ ]:
#Creating a Confusion Matrix
cf_matrix = confusion_matrix(y_validate, y_pred, normalize = "true")
df_cm = pd.DataFrame(cf_matrix, range(2), range(2))
df_cm = df_cm.rename(index=str, columns={0: "Pass Inspection", 1: "Fail Inspection"})
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
##NE: ISN'T THIS FOR TUNING THE LOGISTIC MODEL AND SHOULDN'T IT BE DONE EARLIER TO FIT THE LOGISTIC MODEL? 
#Hyperparameter Tuning
param_grid = {'penalty': ['l1', 'l2', 'elasticnet'],
             'C': np.arange(.1, 1, .1),
               'fit_intercept': ['True', 'False'],
             'solver': ['liblinear', 'saga']}
logit_grid = GridSearchCV(logit_model, param_grid, cv=3)
logit_grid.fit(X_train, y_train)
best_index = np.argmax(logit_grid.cv_results_["mean_test_score"])
best_logit_pred = logit_grid.best_estimator_.predict(X_validate)
print(logit_grid.cv_results_["params"][best_index])
print('Validation Accuracy', accuracy_score(best_logit_pred, y_validate))

### 3.2 Model 2 - Support Vector Machine

In [ ]:
## NE: WHERE IS THE HYPERTUNING FOR SVM? 
#Support Vector Machine Model
svm = SVC()
svm_model = svm.fit(X_train, y_train.ravel())
y_pred = svm_model.predict(X_validate)

svm = svm.SVC(kernel='linear')

In [ ]:
#confusion matrix stuff
cf_matrix = confusion_matrix(y_validate, y_pred, normalize = "true")
df_cm = pd.DataFrame(cf_matrix, range(2), range(2))
df_cm = df_cm.rename(index=str, columns={0: "Pass Inspection", 1: "Fail Inspection"})
#Visualization
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

### 3.2 Model 3 - Decision Tree

In [ ]:
##NE: WHERE IS THE HYPERTUNING FOR THIS? 
#Decision Tree Model
#Creating a decision tree classifier
dt_classifier = tree.DecisionTreeClassifier(criterion='gini',
                       splitter='best',
                       max_depth=5,  # how deep tree nodes can go - I set this to 2... is that okay?
                       min_samples_split=2,
                       min_samples_leaf=1,
                       min_weight_fraction_leaf=0.0,
                       max_features=None,
                       max_leaf_nodes=None,
                       min_impurity_decrease=1e-07,
                       random_state = 12345) #random seed
#Cross Validation
scores = cross_val_score(dt_classifier, X, y, cv=5)
scores
scores.mean()
dt_classifier.fit(X, y)
#Visualization
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(dt_classifier, 
                   feature_names=X.columns,  
                   class_names=["Pass Inspection", "Fail Inspection"],
                   filled=True,
                  fontsize = 10,
                  max_depth = 4)

In [ ]:
#Hypertuning for Decision Tree
param_grid3 = {'min_samples_split': np.arange(2, 10), 'min_samples_leaf': 
np.arange(.05, .2), 'max_leaf_nodes': np.arange(2, 30)}
cv_dtc = GridSearchCV(estimator = dt_classifier, param_grid = param_grid3, cv = 3, 
scoring = accuracy_score, refit='precision', n_jobs=-1)
print(np.arange.cv_results_["params"][best_index])
print('Validation Accuracy', accuracy_score(cv_dtc, y_validate))

### 3.3 Validation Metrics

**Hint**: Try writing a for loop to use [`cross_val_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) to check for accuracy, precision, recall and f1 across all of your models.

### Accuracy
Accuracy is an expression of ratio of correct observations relative to incorrect observations.
Accuracy is defined as: 

$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

In [ ]:
range(len(y_pred))

In [ ]:
## NE: UPDATE THIS ONCE THE EARLIER MODELS HAVE BEEN HYPERTUNED AND DEAL WITH IMBALANCED SAMPLE (MORE PASS THAN FAIL)

#Initialize 4 objects as 0, and then will iterate through vector of predictions to calc our TP, FP, etc. 
#each one of these is in one quadrant of the confusion matrix 
##create list with 4 validation metrics, - run for loop

TP = 0
FP = 0
TN = 0
FN = 0

##looping through for each i, i is a convention for item in a list - so for each item in this list of predicted values 
##put loop inside a function where you write to calc accuracy etc. and feed ypredlogit, ypredsvm, dt_classifier? (take actual and predicted)
##see decision tree notebook - cross_val_score to put all of them together
for i in range(len(y_pred)):
    ##if validate and predict both equal 1 
    if y_validate[i]==y_pred[i]==1:
       TP += 1
    ##we predict it equals 1 but the actual/validate does not equal 1 
    if y_pred[i]==1 and y_validate[i]!=y_pred[i]:
       FP += 1
    ##if validate and prediction equal 0 (if predict 0 and actually is 0)
    if y_validate[i]==y_pred[i]==0:
       TN += 1
    ##if predicted value is 0 and actual/validate is not
    if y_pred[i]==0 and y_pred[i]!=y_validate[i]:
       FN += 1

In [ ]:
accuracy = (TP + TN)/(TP + TN + FP + FN)
print("Accuracy is", accuracy)

### Recall
Recall is defined as:

$$
Recall = \frac{TP}{TP + FN}
$$

In [ ]:
recall = TP/(TP + FN)
print("Recall is", recall)

### Precision
Precision is a measure of how well calibrated predictions are. Precision is defined as: 

$$
Precision = \frac{TP}{TP + FP}
$$

In [ ]:
precision = TP/(TP + FP)
print("Precision is", precision)

### F1 Score
The precision-recall tradeoff can be managed in a few different ways. One popular metric is the F1 score. F1 Score is defined as:

$$
F1 = 2 * \frac{precision * recall}{precision + recall}
$$

In [ ]:
f1 = 2 * (precision * recall)/(precision + recall)
print("F1 Score is", f1)

## 4. Policy Simulation

### 4.1 Interpretable Machine Learning

**Hint**: Use tools like feature importance plots and coefficient plots

In [ ]:
sns.barplot(x="Coefficient", y="Feature", data=logit_data.nlargest(10, 'abs_coef')).set_title("Top Logit Coefficients")
plt.show()

dt_classifier
logit_model 
svm_model 

In [ ]:
logit_model.coef_

##FROM STACKOVERFLOW - MODIFY 
importance = logistic_model.coef_[0]
#importance is a list so you can plot it. 
feat_importances = pd.Series(importance)
feat_importances.nlargest(20).plot(kind='barh',title = 'Feature Importance')

In [ ]:
##EG: ASK KQ IF CAN USE KERNEL LINEAR FOR SVM
svm_model.coef_

In [ ]:
##decision tree classifier feature importance plot 
feat_importances = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(dt_classifier.feature_importances_))], axis = 1)
feat_importances.columns = ["Feature", "Importance"]
sns.barplot(x = "Importance", y = "Feature", data = feat_importances.nlargest(10, 'Importance'))
plt.show()

### 4.2 Prioritize Audits

**Hint**: Look up the [`.predict()`](https://www.kite.com/python/docs/sklearn.linear_model.SGDRegressor.predict), [`.predict_proba()`](https://www.kite.com/python/docs/sklearn.linear_model.LogisticRegression.predict_proba), and [`.sample()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) methods. Then: 
1. Choose one of your models (or train a new simplified model or ensemble!) to predict outcomes and probabilities. 
2. Order your audits by their probability of detecting a "Fail" score
3. Plot your distribution of pass/fail among the first 1000 observations in the dataset
4. Simulate random audits on the full chicago_2011_to_2013.csv dataset by picking 1000 observations at random

### 4.3 Predict on Data with Unseen Labels

In [ ]:
# Fill in the code below with the X data you used for training
X_test = chicago_inspections_2014[chicago_inspections_2014.columns & ....columns]

## 5. Discussion Questions

### 5.1 *Why do we need metrics beyond accuracy when using machine learning in the social sciences and public policy?*

### 5.2 *Imagine that establishments learned about the algorithm being used to determine who gets audited and they started adjusting their behavior (and changing certain key features about themselves that were important for the prediction) to avoid detection. How could policymakers address this interplay between algorithmic decisionmaking and real world behavior?* 